<a href="https://colab.research.google.com/github/peppertaco/AI/blob/main/TavernOAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OAI 태번 실행기

In [ ]:
#@title Tap this if you play on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the music player to keep the tab alive, then start TavernAI below (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title TavernAI
#@markdown **<font color = red><--눌러 (≖ ‸ ≖ ✿)**
#@markdown ###### ***옵션. NGROK 사용 ( 입력시 자동실행 )***
#@markdown - loca.lt 링크보다 ngrok 로딩이 더 빠름 
#@markdown - https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_token = "" #@param {type:"string"}
Model = "GPT4 scale" #@param ["GPT4 scale","GPT4 scale swipe","1.3", "v0.2", "v0.35"] {allow-input: false}
#@markdown - <font color = red>v0.35 무한리롤 버그 있습니다. 무한 리퀘스트 발동으로 토큰 쭉쭉 빠져나감. 사용에 주의하세요 (코랩만 해당됨)
use_google_drive = True #@param {type:"boolean"}
!nvidia-smi
!npm install -g localtunnel
import subprocess
import time
import sys
import os
import threading
import shutil
from google.colab import drive

if NGROK_token:
  !pip install pyngrok
  from pyngrok import ngrok
  os.environ['NGROK_token'] = NGROK_token
  !ngrok authtoken $NGROK_token
  !streamlit run app.py&>/dev/null&
  publ_url = ngrok.connect(addr = '8000')
else:
  publ_url = ''
  print("NGROK 미사용")
    
    
if use_google_drive:
  drive.mount('/content/drive/')
  if not os.path.exists("/content/drive/MyDrive/TavernAI/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats/")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

def copy_characters(use_google_drive=False):
  if not use_google_drive:
    return
  
  src_folder = "/TavernAIColab/public/characters"
  dst_folder = "/content/drive/MyDrive/TavernAI/characters"

  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)

    if os.path.exists(dst_file):
      print(f"{dst_file} already exists. Skipping...")
      continue

    shutil.copy(src_file, dst_folder)
    print(f"{src_file} copied to {dst_folder}")



#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v

if Model == "1.3":
  !git clone https://github.com/peppertaco/TavernAI TavernAIColab
elif Model == "v0.2":
  !git clone https://github.com/peppertaco/TavernAIColab
elif Model == "v0.35":
  !git clone -b new https://github.com/peppertaco/TavernAIColab
elif Model == "GPT4 scale":
  !git clone -b new4 https://github.com/peppertaco/TavernAIColab  
elif Model == "GPT4 scale swipe":
  !git clone -b new4-swipe https://github.com/peppertaco/TavernAIColab  

copy_characters(use_google_drive)
%cd TavernAIColab
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
if use_google_drive:
  %env googledrive=2
!nohup node server.js &
time.sleep(3)
print('')
print('###TavernAI LINK###')
print(publ_url)
!lt --port 8000